In [ ]:
# some self variables that you could use in the real environment class
class SuperEnv(gym.Env):
    
    # Define your action and observation spaces. You want to take actions in one dimension, changing the h42 phase offset,
    # and want a 4D observation, consisting of your relative bunch lengths and bunch intensities. Both your observations and 
    # actions will be should be normalized to be within [-1,1] for optimisation reasons.
    
    metadata= {'render.modes': ['human']}
    
    action_space = gym.spaces.Box(
                                low = np.array([-1,]),#__, # Fill in the __ with your action settings.
                                high = np.array([1,]),#__,
                                shape=(1,),
                                dtype=np.float32)


    ### Define what the observations are to be expected
    observation_space = gym.spaces.Box(
                                low = np.array([-1,-1,-1,-1]), # Fill in the __ with your observation settings.
                                high = np.array([1,1,1,1]),
                                shape=(4,),
                                dtype=np.float32)
    
    ### Stop criteria (constituting "good" splittings with provided difference estimate).
    BUNCH_LENGTH_INT_CRITERIA = 0.02 # Empirically evaluated diff_estimate that constitutes a "good" bunch splitting. Lower means longer training time, but smaller spread in bunch lengths/intensities.
    

    
    

    def __init__(self,
                max_steps = 100,
                max_step_size = 20,
                min_setting = -45,
                max_setting = 45,):
        
        ### Assign hyperparameter settings to attributes
        self.max_steps = max_steps
        self.max_step_size = max_step_size
        self.min_setting = min_setting
        self.max_setting = max_setting
        
        ### Status of the iterations
        # Steps, initializing lists to store actions/states/rewards...
        
        self.counter = 0 # Counts the number of steps taken in the environment!
        self.curr_step = -1  ## Not used ?
        self.phase_correction = 0

        # Initialize lists for tracking episodes
        self.curr_episode = -1
        self.action_episode_memory = []
        self.diff_estimate_memory = []
        self.state_memory = []
        self.phase_set_memory = []
        self.reward_memory = []
        self.is_finalized = False
    
    def step(self, action):
        """
        One step/action in the environment, returning the observable
        and reward. 

        Stopping conditions: max_steps reached, or splitting good enough.
        """
        success = False    
        ################################################################################################################################################################
        # Implement the code below!!
        ################################################################################################################################################################
        
        # Hint: Number of steps taken in environment is tracked in self.counter
        
        
        self._take_action(action) # Actually take action: Define your _take_action function below!! 
        reward = self._get_reward() # Get your reward: Define your _get_reward function below!! Returns a reward value.
        
        state = self.state
        ### Check exit criteria: Achieved good enough state, or taken too many steps.
        ### BUNCH_LENGHT_INT_CRITERIA based on the given diff_estimate value.
        
        if abs(self.diff_estimate) < self.BUNCH_LENGTH_INT_CRITERIA: # Check if the diff_estimate is below criterion. If so, episode finalized and a success!
            self.is_finalized = True 
            success = True
        # print(self.counter)    
        if self.counter >= self.max_steps: # Check if you have exceeded the maximum step limit. If so, episode finalized but not a success...
            self.is_finalized = True

        # Here you can add any extra info you would like to be returned on each step, e.g. episode steps, rewards, actions etc.
        info = {'success': success, 'steps': self.counter, 'profile': self.profile} 
        
        ################################################################################################################################################################
        # Implement the code above!!
        ################################################################################################################################################################
        
        
        return state, reward, self.is_finalized, info # Standardized output according to gym framework.

    def _take_action(self, action):
        """
        Actual action funtion.

        Action from model is scaled to be between [-1,1] for better optimization performance. 
        Converted back to phase setting in degrees using self.max_step_size.
        
        Args:
            action (ndarray): n-dimensional action. Datatype, dimension, and value ranges defined in self.action_space.
        """
        ################################################################################################################################################################
        # Implement the code below!!
        ################################################################################################################################################################

        converted_action = action*self.max_step_size # Convert your action from your normalized range [-1,1] back into degrees using self.max_step_size. HINT: action*self.max_step_size = what?
        self.phase_correction += converted_action # Phase correction tracks previous actions taken to get the cumulative change from start.

        # Phase offset as action, add offset to current phase_set to get next setting. This is what defines which simulated datapoint to collect in your _get_state function!!!
        self.phase_set += converted_action

        # Update the self.state parameter with the new state. The preprovided self._get_state() will provide you with a state based on the current self.phase_set attribute value.
        # The state will consist of a vector of 4 values: [bunch_length_1, bunch_length_2, bunch_intensity_1, bunch_intensity_2].
        self.state = self._get_state()


        ################################################################################################################################################################
        # Implement the code above!!
        ################################################################################################################################################################  
        curr_state = self.state.copy()
        curr_phase_set = self.phase_set.copy()
        self.action_episode_memory.append(action)
        self.state_memory.append(curr_state)
        self.phase_set_memory.append(curr_phase_set)
        
        self.counter += 1

     
    
    def _get_state(self):
        '''
        Get the observable for a given phase_set. This function is provided completed to help you collect datapoints from the pre-simulated dataset.
        The data_class class is written to provide datapoints from a quadsplit dataset, but by always providing h84=0 we only vary the first phase
        offset.

        Comment: The edge cases of trying to move to datapoints outside the simulated dataset needs to be handled.
        Currently it is simply checked whether the phase setting is above the max setting or below the min setting,
        and if so a pre-defined dummy observation is presented. The important factor to consider is to make sure that
        all edge cases are covered by some dummy state, and that the dummy states are unique (so the model can learn
        what steps to take to get back in the right search area). It is also highly advised to give an additional penalty
        in the reward if the agent steps outside our region of simulated data.
        '''
        
        ### Check whether we are within simulated settings
        if (self.phase_set<self.min_setting):
            state = np.array([0.5, -0.5, 0.5, -0.5])
        elif (self.phase_set>self.max_setting):
            state = np.array([-0.5, 0.5, -0.5, 0.5])
        else:
            
            ################################################################################################################################################################
            # Implement the code below!!
            #########################################################################################################################
            
            # Collecting the simulated datapoint, calculating state description
            profile = data_class.get_interpolated_profile(self.phase_set) # Second phase does not affect the first. Since we only care about h42, no need to assign h84 offfset.
            state = get_state_from_profile(profile)
            
            ################################################################################################################################################################
            # Implement the code above!!
            ################################################################################################################################################################
        
            self.profile = profile # Add a tracking of the profile for plotting purposes!
        return state.astype(np.float32)
    
    def _get_reward(self):
        """ Evaluating the reward from the observable/state. 
            The example reward 'simple_profile' is provided, and is based on the final profile
            after the splitting. This is using more information than just the state provided
            to the agent.

            Feel free to experiment and design your own reward as well! 

        Returns:
            float: The reward based on the current state. 
        """ 
        
        
        
        # The shape of your observable should match your optimization_space. For this excercise, it is expected
        # that you use an observation space of a vector with four values
        observable = self.state

        diff_estimate = get_diff_estimate_from_state(observable)
        self.diff_estimate = diff_estimate

        ################################################################################################################################################################
        # Implement the code below!!
        ################################################################################################################################################################
        """ 
         Define your own reward here. The diff_estimate provided above is provided 
         as a simple way to define the difference between your bunches length/intensity 
         after the splitting. You want this to be as small as possible, so a better reward
         should be given for a smaller diff_estimate. There is already a pre-defined
         criterion for the diff_estimate to be considered a "good" splitting provided in
         the BUNCH_LENGTH_INT_CRITERIA attribute. 
         """
        
        #step-wise sparse reward: Reward reaching good conditions fast, steps that do not significantly
        # improve our state give no additional reward.
        
        reward = -1 # Every step is bad: negative reward is the baseline.

        if (self.phase_set<self.min_setting) or (self.phase_set>self.max_setting):
                reward += -4 # If outside our region of interest, give large penalty.
            
        elif abs(diff_estimate) < self.BUNCH_LENGTH_INT_CRITERIA: # Tested to see where it is close to optimal setting
            reward += 101 # If we reach our goal, the agent is heavily incentivized.
        elif abs(diff_estimate) < 0.025:
            reward += 0.75
        elif abs(diff_estimate) < 0.2:
            reward += 0.5
        elif abs(diff_estimate) < 0.5:
            reward += 0.25
        #reward = -diff_estimate #______ # Define your own reward here!! 

        ################################################################################################################################################################
        # Implement the code above!!
        ################################################################################################################################################################
        
        ### Tracking of diff_estimate. Lets you use my render() function to observe your agent.
        curr_diff_estimate = self.diff_estimate.copy()
        self.diff_estimate_memory.append(curr_diff_estimate)
        self.reward_memory.append(reward)
            
        return reward
       
    def reset(self):
        """
        Reset to a random state to start over a training episode.
        This function will be called everytime an episode is started
        to provide an initial state.
        
        Returns:
            state: The inital state of the environment. Should match the shape defined in observation_space.
        """
        # Resetting to start a new episode
        self.curr_episode += 1
        self.counter = 0
        self.is_finalized = False # This tracks wether an episode is complete or not.
        
        #print(f'Resetting for episode {self.curr_episode}')

        # Initializing episode lists to track data for individual episodes. Some used for rendering.
        self.action_episode_memory = []
        self.state_memory= []
        self.phase_set_memory= []
        self.reward_memory= []
        self.diff_estimate_memory= []
        
        ################################################################################################################################################################
        # Implement the code below!!
        ################################################################################################################################################################
        
        # Getting initial state
        
        # Initialize random starting phase_set within [self.min_setting, self.max_setting], for example using random.uniform(min,max)
        self.phase_set = random.uniform(self.min_setting,
                                            self.max_setting)
        
        self.initial_offset = np.copy(self.phase_set)
        self.phase_correction = 0
                                        
        self.state = self._get_state() # call _get_state to get the initial state from the starting phase_set.
        state = self.state
        
        ################################################################################################################################################################
        # Implement the code above!!
        ################################################################################################################################################################
 
        ### Some tracking of state, phase, reward, diff_estimate. Lets you use my render() function to observe your agent.
        self.state_memory.append(state)
        self.phase_set_memory.append(self.phase_set)
        reward = self._get_reward()
        self.reward_memory.append(reward)
        
        return state.astype(np.float32)

    def seed(self, seed=None):
        """
        Set the random seed. Useful if you want to standardize trainings.
        """
        
        random.seed(seed)
        np.random.seed
        
    def render(self, mode='human'):
        
        """
        Rendering function meant to provide a human-readable output. Base function in gym
        environments to override. I provide a simple version that should let you observe 
        your trained agent during evaluation.
        """
        plt.figure('Agent')
        plt.clf()
        plt.subplot(131)
        plt.suptitle(f'Episode {self.curr_episode}')
        plt.title('Current profile')
        plt.plot(self.profile,'b')
        plt.subplot(132)
        plt.title('Difference estimate')
        plt.plot(self.diff_estimate_memory, 'o-')
        plt.axhline(y=self.BUNCH_LENGTH_INT_CRITERIA, color='k', linestyle='--')
        plt.subplot(133)
        plt.title('h42 phase offset')
        plt.plot(np.asarray(self.phase_set_memory, dtype=object), 'go-')
        plt.axhline(y=0, color='k', linestyle='--')
        plt.ylim((-30,30))
        
        #plot_finish(fig=fig, axes=axes, xlabel='Setting', ylabel='Observable')
        plt.pause(0.2)
        

In [ ]:
# Train a model

# Separate evaluation env
eval_env = SuperEnv(max_step_size=45)

# Use deterministic actions for evaluation
eval_callback = EvalCallback(eval_env,  best_model_save_path='./RL_logs/SAC/Super_lims_45',
    log_path='./RL_logs/', eval_freq=1000,
    deterministic=True, render=False)


env = SuperEnv(max_step_size=45)

seed = 7
np.random.seed(seed)
env.seed(seed)
log_dir = './'


ent_coef = 1/5#'auto' # Hyperparameter of SAC, with 'auto' it is also learned during training.

# Define your model
sac_model_super = SAC("MlpPolicy", env, verbose=1, ent_coef=ent_coef, learning_starts=100, tensorboard_log="./hands-on_rl_tensorboard")
print("Starting training...")
sac_model_super.learn(total_timesteps=100000, callback=eval_callback, log_interval=1, tb_log_name="hands_on_rl_SAC_super_45")
sac_model_super.save('./saved_models/RL_agent_super_SAC')
print("Completed training")

In [ ]:
test_environment = SuperEnv()

# Set the environment to different states in our observation space, and check the value of the reward.
# We will also plot our difference estimate, to see that our reward are reasonable in relation to our
# bunch characteristics.

phases = np.linspace(-45,45,361)
rewards = []
diff_estimates = []

for phase in phases:
    test_environment.reset() # Initialize 
    test_environment.phase_set = phase # manually change phase setting
    test_environment.state = test_environment._get_state() # Manually update the state based on current self.phase_set
    reward = test_environment._get_reward() # Manually calculate reward based on current self.state
    
    rewards.append(reward)
    diff_estimates.append(test_environment.diff_estimate) # Also track difference estimates for visited states.

# Plotting

ax1= plt.subplot(211)
plt.title('Reward/diff_estimate given at different phase offsets')
plt.plot(rewards, 'r')
plt.ylabel('Received reward')
ax2 = plt.subplot(212, sharex=ax1)
plt.plot(diff_estimates)
plt.ylabel('Difference estimate')
plt.xticks(np.linspace(0,361,5), np.linspace(-45,45,5))
plt.xlabel('Phase offset [deg]')